# Optuna Trials For Baseline Model

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict, deque
import matplotlib.pyplot as plt
import datetime 

import sys
import time
import tqdm
import pickle
import os

import optuna

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
from torchsummary import summary
from sklearn.model_selection import train_test_split


if os.path.exists('/workspace/data'):
    # Load the dictionary of DataFrames from the pickle
    data_path = '/workspace/data/'
else:
    data_path = '../data/'


In [4]:
class Model(nn.Module):
    def __init__(self, in_features, out_features):
        """Initializes the model layers.

        Args:
            in_features (int): The number of input features of the dataset.
            out_features (list): The number of units in each linear layer.
        """
        num_layers = len(out_features)
        input_dropout = .2
        dropout = .3
        layers = []
        
        layers.append(nn.Dropout(input_dropout))
    
        for i in range(num_layers):
            layers.append(nn.Linear(in_features, out_features[i]))
            layers.append(nn.BatchNorm1d(out_features[i]))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(dropout))
            in_features = out_features[i]
        
        # Binary classification with loss function BCEWithLogitsLoss
        layers.append(nn.Linear(in_features,1))
        
        self.sequential = nn.Sequential(*layers)
        
    def forward(self, x):
        x = self.sequential(x)
        return x

In [5]:
num_workers = 16
batch_size = 8

def prepare_data_loaders(batch_size=batch_size, num_workers=num_workers):
    
    loaders = {
        "train": DataLoader(batch_size=batch_size, num_workers=num_workers, shuffle=True, pin_memory=True, persistent_workers=True),
        "test": DataLoader(batch_size=batch_size, num_workers=num_workers, shuffle=True, pin_memory=True, persistent_workers=True),
        "val": DataLoader(batch_size=batch_size, num_workers=num_workers, shuffle=True, pin_memory=True, persistent_workers=True),
    }
    return loaders

In [6]:
def train_epoch(model, loaders, criterion, optimizer, num_epochs, epoch, device):
    model.train()
    train_loader_tqdm = tqdm(loaders['train'], desc=f'Epoch {epoch+1}/{num_epochs}', unit='batch')
    running_loss = deque(maxlen=10000)
    
    # Train epoch
    for X_train, y_train in train_loader_tqdm:
        optimizer.zero_grad()
        
        X_train_gpu = X_train.to(device)
        y_train_gpu = y_train.to(device)
        
        output_gpu = model(X_train_gpu)
        
        loss = criterion(output_gpu, y_train_gpu)
        running_loss.append(loss.item())
        
        loss.backward()
        optimizer.step()
        
        train_loader_tqdm.set_postfix(f"loss={running_loss / len(running_loss)}")
        
    return
    

def test_model(model, loaders, criterion, device, num_epochs, epoch, loader='test'):
    # Validate epoch:
    model.eval()
    test_loader_tqdm = tqdm(loaders[loader], desc=f'Epoch {epoch+1}/{num_epochs}', unit='batch')
    test_loss = []
    num_tested = []
    correct_pred = []
    
    for X_test, y_test in test_loader_tqdm:
        X_test_gpu = X_test.to(device)
        y_test_gpu = y_test.to(device)
        
        output_gpu = model(X_test_gpu)
        
        test_loss.append(criterion(output_gpu, y_test_gpu).item() * X_test.shape[0])
        num_tested.append(X_test.shape(0))
        
        correct_pred += torch.sum(((nn.Sigmoid(output_gpu) > .5) == y_test_gpu))
        
        test_loader_tqdm.set_postfix(f"loss={sum(test_loss) / sum(num_tested)}, acc={correct_pred / sum(num_tested)}")
    
    return sum(test_loss) / sum(num_tested), correct_pred / sum(num_tested)


In [7]:
def objective(trial, num_layers, min_out, max_out, in_features, loaders, study_name):
    # Generate the output features for each layer using trial suggestions
    out_features = []
    for i in range(num_layers):
        out_features.append(trial.suggest_int(f"out_features_layer_{i}", min_out, max_out))
    
    # Set device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # Create model and move to device
    model = Model(in_features, out_features).to(device)
    
    # Initialize optimizer and loss function
    optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)
    criterion = nn.BCEWithLogitsLoss()
    
    num_epochs = 3
    # Training loop for num_epochs
    for epoch in range(num_epochs):
        train_epoch(model, loaders, criterion, optimizer, num_epochs, epoch, device)
        test_loss, test_accuracy = test_model(model, loaders, criterion, device, num_epochs, epoch, loader='test')

    # Return the test loss to be minimized
    return test_loss


In [ ]:
loaders = prepare_data_loaders()

# Define the parameters for the study
study_name = "Baseline"               # Name of your study
num_layers = 5                        # Example number of layers
min_out = 32                          # Minimum number of units per layer
max_out = 512                         # Maximum number of units per layer
in_features = 128                     # Example input feature size

study = optuna.create_study(study_name=study_name, direction='minimize')

# Define the objective function and run the optimization
study.optimize(lambda trial: objective(trial, num_layers, min_out, max_out, in_features, loaders, study_name), 
               n_trials=5, show_progress_bar=True)  # You can specify how many trials you want

# Print the best parameters found by the study
print(f"Best parameters: {study.best_params}")
print(f"Best trial: {study.best_trial}")



